**By B08705027 資管四 林暐倫**

In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# 1. Finetuning BERT for our dataset

In [ ]:
# !pip install transformers
# !pip install torch

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
# Read Files to List
doc_num = 91
question_doc = []
choice_doc = []
ans_doc = []

for i in range(1, doc_num + 1):
    try:
        with open(f'Data/New/{i}.txt', encoding='utf-8') as f:
            raw_q = f.read()
    except:
        with open(f'Data/New/{i}.txt', encoding='big5') as f:
            raw_q = f.read()
    question_doc.append(raw_q)

In [38]:
question_doc[:1]

['Fiction is the name we use for stories that are make-believe, such as Harry Potter or Alice in the Wonderland. But fiction isn’t always different from the way things usually are It can be so close to the truth that it seems as real as something that happened to you this morning. Or, fiction can be as fantastic as the most unbelievable fairy tale. Not everything in a fictional story has to be made up. You could write a story in which you fly to the moon. You, of course, are real, and the moon is real, and many of the things that you could describe, such as the stars, the wind, and the pull of gravity, would be real. But your trip through space would be fiction. It would be a trip you took in your imagination. Nonfiction, on the other hand, is all about true things. Nothing is made up. Someone’s biography is nonfiction; so is your autobiography. So are articles in your local newspaper, and school reports on science. History is nonfiction, too. Imagine writing history about the 1989 San

finetuing BERT w question_doc

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:

from nltk import tokenize
text = []
for question in question_doc:
  temp = tokenize.sent_tokenize(question)
  # temp = tokenizer.enocde_plus(question)
  temp.append(' ') # append space as sep
  for to_merge in temp:
    text.append(to_merge)

# preprocessing to remove number

In [39]:
bag = text
bag_size = len(bag)
print(bag_size)


1792


In [40]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != ''
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

In [41]:
for i in range(2):
    print(label[i])
    print(sentence_a[i] + '\n---')
    print(sentence_b[i] + '\n')


1
An old proverb says, “Truth is stranger than fiction
---
His name is Bond.

1
I need directions home
---
The practice then spread to other towns as a common custom.



In [42]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')


In [43]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [44]:
inputs.token_type_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
inputs['labels'] = torch.LongTensor([label]).T
inputs.labels[:10]

tensor([[1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [0]])

In [ ]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = MeditationsDataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

Training setup

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)


BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-6)


In [ ]:
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 10

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

Epoch 9: 100%|██████████| 4/4 [02:46<00:00, 41.54s/it, loss=0.0348]


In [ ]:
PATH = "new_model.pt"
torch.save(model.state_dict(), PATH)

# 2. Use finetuned BERT to predict NSP (bool-based)

In [46]:
choice_cols = ['A', 'B', 'C', 'D', 'E', 'F']
choice_df = pd.DataFrame(choice_doc, columns=choice_cols, index=[i + 1 for i in range(len(choice_doc))])
for col in choice_cols:
    choice_df[col] = choice_df[col].str.replace(r'\([A-F]\)', '')

ans_cols = [1, 2, 3, 4, 5]
ans_df = pd.DataFrame(ans_doc, columns=ans_cols, index=[i + 1 for i in range(len(choice_doc))])

display(choice_df.head())
display(ans_df.head())

,A,B,C,D,E,F


,1,2,3,4,5


In [ ]:
count_NotNextSentence = 0
NotNextSentence = []

for i in range(len(question_doc)): # each document
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  for j in range(5): # each question
      for k in range(6): # each option
        if (choice_df.iloc[i][choice_cols[k]] != 'None'):
          inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
          outputs = model(**inputs)
          if (torch.argmax(outputs.logits).item() == 1): # NotNextSentence
            count_NotNextSentence += 1
            NotNextSentence.append([i+1, j+1, choice_cols[k]]) # Doc i+1, Question j+1, Choice choice_cols[k], is not the answer
  print("=== Finish Doc", i+1, "===")


=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [ ]:
# In all situations (91*5*5 = 2375), only `count_NotNextSentence` situations are defined as `NotNextSentence`
print(count_NotNextSentence)

333


In [ ]:
print(NotNextSentence)

[[1, 2, 'F'], [1, 3, 'B'], [1, 3, 'F'], [1, 4, 'F'], [1, 5, 'F'], [2, 1, 'F'], [2, 2, 'F'], [2, 3, 'F'], [2, 4, 'F'], [3, 4, 'D'], [3, 5, 'D'], [3, 5, 'F'], [4, 1, 'D'], [4, 2, 'C'], [4, 2, 'D'], [4, 2, 'E'], [4, 2, 'F'], [4, 3, 'B'], [4, 4, 'F'], [5, 2, 'F'], [5, 3, 'A'], [5, 5, 'F'], [6, 1, 'B'], [6, 1, 'F'], [6, 2, 'F'], [6, 4, 'F'], [6, 5, 'F'], [7, 1, 'D'], [7, 2, 'F'], [7, 3, 'F'], [7, 4, 'F'], [7, 5, 'F'], [8, 1, 'F'], [8, 5, 'D'], [9, 4, 'A'], [9, 4, 'F'], [10, 1, 'A'], [10, 1, 'C'], [10, 1, 'D'], [10, 3, 'F'], [10, 4, 'F'], [13, 2, 'C'], [14, 1, 'C'], [14, 2, 'C'], [14, 3, 'C'], [14, 4, 'C'], [17, 5, 'F'], [18, 5, 'D'], [19, 1, 'B'], [19, 1, 'C'], [19, 1, 'E'], [19, 2, 'C'], [19, 4, 'B'], [20, 2, 'C'], [20, 3, 'C'], [22, 1, 'A'], [22, 1, 'B'], [22, 1, 'D'], [22, 1, 'F'], [22, 2, 'B'], [22, 2, 'F'], [22, 3, 'B'], [22, 3, 'E'], [22, 3, 'F'], [22, 4, 'A'], [22, 4, 'D'], [22, 4, 'E'], [22, 5, 'A'], [22, 5, 'C'], [22, 5, 'E'], [23, 1, 'B'], [23, 1, 'F'], [23, 2, 'E'], [23, 2, 'F'],

# 3. Use finetuned BERT to predict NSP (probability-based)

In [ ]:
# Read Files to List
doc_num = 91
question_doc = []
choice_doc = []
ans_doc = []

for i in range(1, doc_num + 1):
    try:
        with open(f'Data/Question/{i}.txt', encoding='utf-8') as f:
            raw_q = f.read()
    except:
        with open(f'Data/Question/{i}.txt', encoding='big5') as f:
            raw_q = f.read()
    try:
        with open(f'Data/Choice/{i}.txt', encoding='utf-8') as f:
            raw_ch = f.read()
    except:
        with open(f'Data/Choice/{i}.txt', encoding='big5') as f:
            raw_ch = f.read()
    try:
        with open(f'Data/Answer/{i}.txt', encoding='utf-8') as f:
            raw_a = f.read()
    except:
        with open(f'Data/Answer/{i}.txt', encoding='big5') as f:
            raw_a = f.read()
    question_doc.append(raw_q)
    choice_doc.append(raw_ch.split('\n'))
    ans_doc.append(list(raw_a))

In [ ]:
choice_cols = ['A', 'B', 'C', 'D', 'E', 'F']
choice_df = pd.DataFrame(choice_doc, columns=choice_cols, index=[i + 1 for i in range(len(choice_doc))])
for col in choice_cols:
    choice_df[col] = choice_df[col].str.replace(r'\([A-F]\)', '')

ans_cols = [1, 2, 3, 4, 5]
ans_df = pd.DataFrame(ans_doc, columns=ans_cols, index=[i + 1 for i in range(len(choice_doc))])

display(choice_df.head())
display(ans_df.head())

,A,B,C,D,E,F
1,Nothing is made up.,"History is nonfiction, too.",But your trip through space would be fiction.,You could write a story in which you fly to t...,But fiction isn’t always different from the w...,None
2,"But when I pulled into the driveway, there wa...",Hours later I called my daughter and asked if...,"One time, I even shouted at Derek when he unp...",Derek became part of our life and seemed to f...,"Without any experience in raising pets, my hu...",None
3,Another genre commonly found in Chinese brush...,"However, the subject matters later expanded b...","As a result, they have obtained more natural ...",Its growth has inevitably reflected the chang...,It then gradually developed into two separate...,
4,She got to look at the fine old houses as wel...,Architects make drawings and careful plans of...,The word for what she does is rehabilitation.,Then she went on to study architecture.,That is precisely what Carrie does.,None
5,There are definitely two sides to this issue.,Scores decreased as the family size increased...,An intelligence test was administered to over...,"On the other side are Rutherford and Sewell, ...","Since then, the theory has been elaborated an...",None


,1,2,3,4,5
1,E,D,C,A,B
2,E,D,C,A,B
3,D,B,E,A,C
4,C,E,A,D,B
5,C,B,E,A,D


In [ ]:
from torch.nn.functional import softmax
answer_prob = []

for i in range(len(question_doc)): # each document
  tmp_answer_prob_for_doc = []
  question_sentence = re.split('\(1\)|\(2\)|\(3\)|\(4\)|\(5\)', question_doc[i])
  quesion_num = 5
  if (ans_df.iloc[i][5] == None):
    quesion_num = 4
  for j in range(quesion_num): # each question
    tmp_answer_prob_for_question = []
    for k in range(6): # each option
      if (choice_df.iloc[i][choice_cols[k]] != None):
        # sentence before
        inputs = tokenizer(question_sentence[j], choice_df.iloc[i][choice_cols[k]], return_tensors='pt')
        outputs = model(**inputs)[0]
        probs = softmax(outputs, dim=1)
        # sentence after
        if (len(question_sentence) > j):
          inputs_2 = tokenizer(choice_df.iloc[i][choice_cols[k]], question_sentence[j+1], return_tensors='pt')
          outputs_2 = model(**inputs_2)[0]
          probs_2 = softmax(outputs_2, dim=1)
          tmp_answer_prob_for_question.append((probs[0][0].item() + probs_2[0][0].item()) / 2)
        else:
          tmp_answer_prob_for_question.append(probs[0][0].item())
    tmp_answer_prob_for_doc.append(tmp_answer_prob_for_question)
  answer_prob.append(tmp_answer_prob_for_doc)

  print("=== Finish Doc", i+1, "===")

=== Finish Doc 1 ===
=== Finish Doc 2 ===
=== Finish Doc 3 ===
=== Finish Doc 4 ===
=== Finish Doc 5 ===
=== Finish Doc 6 ===
=== Finish Doc 7 ===
=== Finish Doc 8 ===
=== Finish Doc 9 ===
=== Finish Doc 10 ===
=== Finish Doc 11 ===
=== Finish Doc 12 ===
=== Finish Doc 13 ===
=== Finish Doc 14 ===
=== Finish Doc 15 ===
=== Finish Doc 16 ===
=== Finish Doc 17 ===
=== Finish Doc 18 ===
=== Finish Doc 19 ===
=== Finish Doc 20 ===
=== Finish Doc 21 ===
=== Finish Doc 22 ===
=== Finish Doc 23 ===
=== Finish Doc 24 ===
=== Finish Doc 25 ===
=== Finish Doc 26 ===
=== Finish Doc 27 ===
=== Finish Doc 28 ===
=== Finish Doc 29 ===
=== Finish Doc 30 ===
=== Finish Doc 31 ===
=== Finish Doc 32 ===
=== Finish Doc 33 ===
=== Finish Doc 34 ===
=== Finish Doc 35 ===
=== Finish Doc 36 ===
=== Finish Doc 37 ===
=== Finish Doc 38 ===
=== Finish Doc 39 ===
=== Finish Doc 40 ===
=== Finish Doc 41 ===
=== Finish Doc 42 ===
=== Finish Doc 43 ===
=== Finish Doc 44 ===
=== Finish Doc 45 ===
=== Finish Doc 46 =

In [ ]:
our_answer_1 = []

for i in range(len(answer_prob)):
  num_q = len(answer_prob[i])
  num_o = len(answer_prob[i][0])
  chosen = np.zeros((num_q, num_o)) # chosen[j][k] = question j, option k
  tmp_answer = np.zeros(num_q)
  for x in range(num_q): # we need to find answers for all questions
    # find the highest probability for this round
    pos_q = 0 # record the highest probability position
    pos_o = 0
    highest_prob = 0
    for j in range(num_q):
      for k in range(num_o):
        if (answer_prob[i][j][k] > highest_prob and chosen[j][k] == 0):
          highest_prob = answer_prob[i][j][k]
          pos_q = j
          pos_o = k
    # record the answer
    tmp_answer[pos_q] = pos_o
    for j in range(num_q):
      chosen[j][pos_o] = 1
    for k in range(num_o):
      chosen[pos_q][k] = 1
  our_answer_1.append(tmp_answer)

In [ ]:
correct_rate = []

for i in range(len(our_answer_1)):
  count = 0
  ours = []
  correct = []
  for j in range(len(our_answer_1[i])):
    correct.append(ans_df.iloc[i][j+1])
    if (our_answer_1[i][j] == 0):
      ours.append('A')
    elif (our_answer_1[i][j] == 1):
      ours.append('B')
    elif (our_answer_1[i][j] == 2):
      ours.append('C')
    elif (our_answer_1[i][j] == 3):
      ours.append('D')
    elif (our_answer_1[i][j] == 4):
      ours.append('E')
    elif (our_answer_1[i][j] == 5):
      ours.append('F')
    if (ans_df.iloc[i][j+1] == ours[j]):
      count += 1
  print("Doc", i+1, ":", count/len(our_answer_1[i]), ours, correct)
  correct_rate.append(count/len(our_answer_1[i]))

Doc 1 : 0.2 ['E', 'C', 'D', 'B', 'A'] ['E', 'D', 'C', 'A', 'B']
Doc 2 : 0.6 ['E', 'D', 'C', 'B', 'A'] ['E', 'D', 'C', 'A', 'B']
Doc 3 : 0.6 ['A', 'B', 'E', 'F', 'C'] ['D', 'B', 'E', 'A', 'C']
Doc 4 : 0.4 ['B', 'C', 'A', 'D', 'E'] ['C', 'E', 'A', 'D', 'B']
Doc 5 : 0.4 ['C', 'A', 'E', 'D', 'B'] ['C', 'B', 'E', 'A', 'D']
Doc 6 : 0.0 ['D', 'B', 'C', 'A', 'E'] ['C', 'E', 'A', 'D', 'B']
Doc 7 : 0.0 ['E', 'C', 'B', 'D', 'A'] ['B', 'D', 'A', 'E', 'C']
Doc 8 : 0.6 ['D', 'E', 'A', 'C', 'B'] ['A', 'E', 'D', 'C', 'B']
Doc 9 : 0.2 ['A', 'E', 'C', 'D', 'B'] ['E', 'C', 'D', 'A', 'B']
Doc 10 : 0.4 ['C', 'D', 'A', 'E', 'B'] ['E', 'D', 'A', 'B', 'C']
Doc 11 : 0.4 ['F', 'B', 'A', 'C', 'D'] ['F', 'D', 'A', 'E', 'C']
Doc 12 : 0.6 ['B', 'E', 'D', 'A', 'F'] ['B', 'A', 'D', 'E', 'F']
Doc 13 : 0.4 ['E', 'D', 'F', 'C', 'A'] ['B', 'D', 'F', 'A', 'E']
Doc 14 : 0.2 ['D', 'B', 'F', 'E', 'A'] ['E', 'A', 'F', 'C', 'D']
Doc 15 : 0.2 ['C', 'E', 'B', 'A', 'D'] ['E', 'A', 'B', 'F', 'C']
Doc 16 : 0.4 ['C', 'E', 'A', 'B', 

In [ ]:
np.mean(correct_rate)

0.4021978021978022

In [ ]:
import collections
collections.Counter(correct_rate)

Counter({0.2: 22, 0.6: 20, 0.4: 21, 0.0: 9, 0.8: 1, 0.25: 4, 1.0: 6, 0.5: 8})

In [ ]:
print("1. ", np.mean(correct_rate[0:22]), collections.Counter(correct_rate[0:22]))
print("2. ", np.mean(correct_rate[22:42]), collections.Counter(correct_rate[22:42]))
print("3. ", np.mean(correct_rate[42:62]), collections.Counter(correct_rate[42:62]))
print("4. ", np.mean(correct_rate[62:91]), collections.Counter(correct_rate[62:91]))


1.  0.34545454545454546 Counter({0.4: 7, 0.2: 5, 0.6: 5, 0.0: 4, 0.8: 1})
2.  0.5 Counter({0.5: 8, 1.0: 5, 0.25: 4, 0.0: 3})
3.  0.39999999999999997 Counter({0.6: 7, 0.2: 7, 0.4: 6})
4.  0.3793103448275862 Counter({0.2: 10, 0.6: 8, 0.4: 8, 0.0: 2, 1.0: 1})


# 4. Use finetuned BERT for NSP with linear sum assignment

In [ ]:
from scipy.optimize import linear_sum_assignment
our_answer_2 = []
tmp_prob_for_LSA = answer_prob

for i in range(len(answer_prob)):
  for j in range(len(answer_prob[i])):
    for k in range(len(answer_prob[i][j])):
      tmp_prob_for_LSA[i][j][k] = -tmp_prob_for_LSA[i][j][k]
  row_ind, col_ind = linear_sum_assignment(tmp_prob_for_LSA[i])
  our_answer_2.append(col_ind)
  print(i, col_ind)

0 [0 3 2 1 4]
1 [4 3 2 0 1]
2 [3 1 4 0 2]
3 [1 0 4 2 3]
4 [2 1 4 3 0]
5 [3 4 2 0 1]
6 [1 2 4 3 0]
7 [0 4 3 2 1]
8 [4 2 1 3 0]
9 [1 0 3 4 2]
10 [5 1 0 2 3]
11 [1 0 3 5 4]
12 [0 3 5 2 4]
13 [3 1 4 5 0]
14 [3 4 1 0 5]
15 [5 4 2 1 0]
16 [2 1 3 0 4]
17 [3 2 5 1 4]
18 [5 4 3 0 2]
19 [3 0 4 1 5]
20 [2 1 0 5 4]
21 [4 0 2 1 5]
22 [2 1 0 3]
23 [1 3 0 2]
24 [3 0 1 2]
25 [3 0 1 2]
26 [2 3 0 1]
27 [1 2 0 3]
28 [1 3 2 0]
29 [1 0 3 2]
30 [0 2 3 1]
31 [2 1 0 3]
32 [2 1 0 3]
33 [1 3 2 0]
34 [1 3 0 2]
35 [0 3 1 2]
36 [2 1 3 0]
37 [2 1 3 0]
38 [3 0 1 2]
39 [3 2 1 0]
40 [1 2 3 0]
41 [0 2 3 1]
42 [4 0 2 3 1]
43 [4 0 5 1 2]
44 [2 5 3 4 0]
45 [4 1 5 0 3]
46 [2 4 3 5 0]
47 [5 2 0 1 3]
48 [3 1 0 4 5]
49 [3 4 5 0 2]
50 [1 3 2 5 4]
51 [5 0 2 4 1]
52 [1 0 2 5 3]
53 [1 3 4 0 2]
54 [3 1 5 4 0]
55 [4 2 1 3 5]
56 [2 1 5 0 4]
57 [2 4 3 5 0]
58 [1 2 5 4 0]
59 [2 4 1 3 5]
60 [1 4 0 2 5]
61 [1 5 3 0 4]
62 [1 5 3 4 0]
63 [4 0 3 5 2]
64 [2 4 3 1 5]
65 [3 5 2 1 4]
66 [2 0 5 1 3]
67 [3 1 2 4 5]
68 [5 4 3 2 1]
69 [2 4 0 3 5]


In [ ]:
## Compare our answer with the correct answer
correct_rate = []

for i in range(len(our_answer_2)):
  count = 0
  ours = []
  correct = []
  for j in range(len(our_answer_2[i])):
    correct.append(ans_df.iloc[i][j+1])
    if (our_answer_2[i][j] == 0):
      ours.append('A')
    elif (our_answer_2[i][j] == 1):
      ours.append('B')
    elif (our_answer_2[i][j] == 2):
      ours.append('C')
    elif (our_answer_2[i][j] == 3):
      ours.append('D')
    elif (our_answer_2[i][j] == 4):
      ours.append('E')
    elif (our_answer_2[i][j] == 5):
      ours.append('F')
    if (ans_df.iloc[i][j+1] == ours[j]):
      count += 1
  print("Doc", i+1, ":", count/len(our_answer_2[i]), ours, correct)
  correct_rate.append(count/len(our_answer_2[i]))

Doc 1 : 0.4 ['A', 'D', 'C', 'B', 'E'] ['E', 'D', 'C', 'A', 'B']
Doc 2 : 1.0 ['E', 'D', 'C', 'A', 'B'] ['E', 'D', 'C', 'A', 'B']
Doc 3 : 1.0 ['D', 'B', 'E', 'A', 'C'] ['D', 'B', 'E', 'A', 'C']
Doc 4 : 0.0 ['B', 'A', 'E', 'C', 'D'] ['C', 'E', 'A', 'D', 'B']
Doc 5 : 0.6 ['C', 'B', 'E', 'D', 'A'] ['C', 'B', 'E', 'A', 'D']
Doc 6 : 0.4 ['D', 'E', 'C', 'A', 'B'] ['C', 'E', 'A', 'D', 'B']
Doc 7 : 0.2 ['B', 'C', 'E', 'D', 'A'] ['B', 'D', 'A', 'E', 'C']
Doc 8 : 1.0 ['A', 'E', 'D', 'C', 'B'] ['A', 'E', 'D', 'C', 'B']
Doc 9 : 0.4 ['E', 'C', 'B', 'D', 'A'] ['E', 'C', 'D', 'A', 'B']
Doc 10 : 0.2 ['B', 'A', 'D', 'E', 'C'] ['E', 'D', 'A', 'B', 'C']
Doc 11 : 0.4 ['F', 'B', 'A', 'C', 'D'] ['F', 'D', 'A', 'E', 'C']
Doc 12 : 0.6 ['B', 'A', 'D', 'F', 'E'] ['B', 'A', 'D', 'E', 'F']
Doc 13 : 0.6 ['A', 'D', 'F', 'C', 'E'] ['B', 'D', 'F', 'A', 'E']
Doc 14 : 0.0 ['D', 'B', 'E', 'F', 'A'] ['E', 'A', 'F', 'C', 'D']
Doc 15 : 0.2 ['D', 'E', 'B', 'A', 'F'] ['E', 'A', 'B', 'F', 'C']
Doc 16 : 0.8 ['F', 'E', 'C', 'B', 

In [ ]:
np.mean(correct_rate)

0.5010989010989011

In [ ]:
import collections
collections.Counter(correct_rate)

Counter({0.4: 23, 1.0: 18, 0.0: 6, 0.6: 13, 0.2: 17, 0.8: 4, 0.5: 6, 0.25: 4})

In [ ]:
print("1. ", np.mean(correct_rate[0:22]), collections.Counter(correct_rate[0:22]))
print("2. ", np.mean(correct_rate[22:42]), collections.Counter(correct_rate[22:42]))
print("3. ", np.mean(correct_rate[42:62]), collections.Counter(correct_rate[42:62]))
print("4. ", np.mean(correct_rate[62:91]), collections.Counter(correct_rate[62:91]))


1.  0.49090909090909096 Counter({0.4: 5, 1.0: 4, 0.6: 4, 0.2: 4, 0.0: 3, 0.8: 2})
2.  0.7 Counter({1.0: 10, 0.5: 6, 0.25: 4})
3.  0.45 Counter({0.2: 6, 0.4: 6, 0.6: 3, 0.8: 2, 1.0: 2, 0.0: 1})
4.  0.40689655172413797 Counter({0.4: 12, 0.2: 7, 0.6: 6, 1.0: 2, 0.0: 2})
